In [16]:
import pandas as pd
from itertools import combinations
import os
from tqdm import tqdm

In [5]:
homog = pd.read_csv('/public21/home/sc90258/huangying/analysis/12.pan_mosquito/20240830_homo_gene/mosquito_all_homog.csv')
homog

,aae,alb,cxpip,cxtri
0,LOC110673977,LOC109409628,LOC120416324,Ctri05375
1,LOC110673980,LOC115258594,LOC120427986,Ctri05761
2,LOC110673981,LOC109427304,LOC120415083,Ctri12482
3,LOC110673983,LOC109402949,LOC120425061,Ctri03408
4,LOC110673984,LOC134285458,LOC120428736,Ctri02287
...,...,...,...,...
8167,LOC5580326,LOC109420085,LOC120419188,Ctri05472
8168,LOC5580327,LOC109420086,LOC120419195,Ctri05473
8169,LOC5580329,LOC109420083,LOC120419194,Ctri05470
8170,LOC5580330,LOC109420082,LOC120422788,Ctri05866


In [21]:
def fmt_degs(trt_pth, ctl_pth, trt_sp, ctl_sp, odir, homog=homog):
    trt_df = pd.read_csv(trt_pth)
    if trt_sp != 'aae':
        dct = dict(homog[[trt_sp, 'aae']].values)
        trt_df['gene'] = trt_df['gene'].map(dct)

    ctl_df = pd.read_csv(ctl_pth)
    if ctl_sp != 'aae':
        dct = dict(homog[[ctl_sp, 'aae']].values)
        ctl_df['gene'] = ctl_df['gene'].map(dct)

    trt_df = trt_df[(trt_df['de_coef'] > 0) & (trt_df['de_pval'] <= 0.05)]
    ctl_df = ctl_df[(ctl_df['de_coef'] > 0) & (ctl_df['de_pval'] <= 0.05)]
    ctl_df = ctl_df[~ctl_df['gene'].isin(trt_df['gene'])]
    ctl_df['de_coef'] = ctl_df['de_coef'] * -1

    res = pd.concat([trt_df, ctl_df], axis=0)

    ofname = os.path.basename(trt_pth)
    opth = os.path.join(odir, ofname)
    res.to_csv(opth, index=False)

In [22]:
idir = '2.cmp_res/'
odir = '4.all_degs/'

for (trt, ctl) in combinations(['aae', 'alb', 'cxpip', 'cxtri'], 2):
    trtdir = os.path.join(idir, f'{trt}_vs_{ctl}')
    ctldir = os.path.join(idir, f'{ctl}_vs_{trt}')

    trtfiles = sorted([os.path.join(trtdir, f) for f in os.listdir(trtdir)])
    ctlfiles = sorted([os.path.join(ctldir, f) for f in os.listdir(ctldir)])

    osubdir = os.path.join(odir, f'{trt}_vs_{ctl}')
    if not os.path.exists(osubdir):
        os.mkdir(osubdir)

    for trt_pth, ctl_pth in tqdm(zip(trtfiles, ctlfiles)):
        fmt_degs(trt_pth, ctl_pth, trt, ctl, osubdir)

11it [00:01, 10.13it/s]
11it [00:01, 10.57it/s]
11it [00:01, 10.86it/s]
11it [00:01,  9.38it/s]
11it [00:01, 10.54it/s]
11it [00:01,  9.68it/s]
